# Project 1

The purpose of this notebook to document preliminary analyses of DiscGolf to average income.

DiscGolf.csv was sourced from https://www.kaggle.com/datasets/lanekatris/pdga-united-states-disc-golf-courses

postcode_level_averages was sourced from https://www.kaggle.com/datasets/hamishgunasekara/average-income-per-zip-code-usa-2018

Data joined and saved to the Resources folder as "draft_dgolf_avincome.csv"


In [ ]:
#Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
from scipy.stats import linregress
import datetime

In [ ]:
# #read CSV and create Pandas DataFrames
# draft_df = pd.DataFrame(pd.read_csv('Resources/draft_dgolf_avincome.csv'))
# draft_df.head()

In [ ]:
# check df
# draft_df.columns("Course ID", "Course Name", "City", "State", "Zip Code", "Hole Count",
#                  "Rating", "Latitude", "Longitude", "Modified", "Comments", "State Abbreviation",
#                  "Zip", "Total Population", "Total Income", "Country", "Average Income")
# draft_df

In [ ]:
# bob = group_df['total_pop'].sum()
# bob

In [ ]:
# subdraft_df = draft_df.loc[draft_df['state_x']]
# subdraft_df.head()

In [ ]:
# #create summary by state:  of number of courses, total population, average income, course rating

# subdraft_df = draft_df.loc['state_x','total_pop','avg_income','holeCount','rating']
# # subdraftdf.head()
# group_df = subdraft_df.groupby(['state_x'])

# ltotpopX = pd.DataFrame(group_df['total_pop'].sum())
# lavinc = group_df['avg_income'].mean()
# lavholes = group_df['holeCount'].mean()
# lavrating = group_df['rating'].mean()
# ltotcourses = group_df['holeCount'].count()

# # totpop = pd.DataFrame(ltotpop).transpose()
# # avinc = pd.DataFrame(lavinc).transpose()
# # avholes = pd.DataFrame(lavholes).transpose()
# # avrating = pd.DataFrame(lavrating).transpose()
# # totcourses = pd.DataFrame(ltotcourses).transpose()

# # merge12 = pd.merge(totpop,avinc,on='state_x')
# # merge34 = pd.merge(avholes,avrating,on='state_x')
# # merge12nmore = pd.merge(merge12,totcourses,on='state_x').rename(columns = {"holeCount":"n_courses"})
# # merge12nmore
# # mergeall = pd.merge(merge12nmore,merge34,on='state_x')
# # mergeall.head()

In [ ]:
# ltotpopX

In [ ]:
# mergeallsort = mergeall.sort_values('n_courses',ascending= False)
# mergeallsort

In [ ]:
# mergeall.plot.scatter(x = 'n_courses',y = 'total_pop',s = 50)

In [ ]:
# mergeall.plot.scatter(x = 'n_courses',y = 'avg_income',s = 50)

In [ ]:
# mergeall.plot.scatter(x = 'n_courses', y = 'rating',s = 50)

In [ ]:
# #create a scatterplot function
# # df = 'mergeall'
# # datasetID = 'DiscGolf'
# # xparam = 'ltotcourses'
# # yparam = 'ltotpop'
# # yunits = 'state_pop'
# fcolor = 'lightblue'
# ecolor = 'mediumblue'

# #some color options mistyrose and maroon,seashell  and saddlebrown,honeydew and forestgreen,lightblue and meduimblue
# #https://matplotlib.org/stable/gallery/color/named_colors.html
# # x_value = []
# # y_value = []

# def LRPlots(x_value, y_value, x_label, y_label):
#     plt.scatter(x_value, y_value, marker="o", facecolors=fcolor, edgecolors=ecolor, s=30, alpha=0.75)
#     plt.title(f"{x_value} vs. {y_value}")
#     plt.xlabel(x_label)
#     plt.ylabel(y_label)
#     plt.grid(True)
    
#     (slope, intercept, rvalue, pvalue, stderr) = linregress(x_value, y_value)
#     line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    
#     regress_values = x_value * slope + intercept
    
#     plt.plot(x_value, regress_values, "r-")
    
#     plt.annotate(line_eq, (x_value.mean(), y_value.mean()), fontsize=15, color="red")
    
#     print(f"The r-value is: {rvalue}")
    
    
    
# #     xdata = mergeallsort[xparam]
# #     ydata = mergeallsort[yparam]

# #     (slope, intercept, rvalue, pvalue, stderr) = linregress(xdata, ydata)
# #     regress_values = xdata * slope + intercept
# #     line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# #     plt.figure(figsize = (9,7))
# #     plt.scatter(xdata, ydata, marker="o", facecolors=fcolor, edgecolors=ecolor,s =75)

# #     # plt.xlim(-55,85)
# #     # plt.ylim(0,105)
# #     annotatex = min(xdata)+0.1*max(xdata)
# #     annotatey = max(ydata)-0.1*max(ydata)

# #     plt.plot(xdata,regress_values,"darkslategray")
# #     plt.annotate(f'{line_eq} r={round(rvalue,2)}',(annotatex,annotatey),fontsize=15,color="black")
    
# #     plt.title(f"{xparam} vs. {yparam}")
# #     plt.xlabel(f"{xparam}")
# #     plt.ylabel(f"{yparam} {yunits}")
# #     plt.grid()
    
# #     print(f'The r-value is {round(rvalue,2)}')

# #     # Save the figure
# #     figname = f'{datasetID}_{xparam}_vs_{yparam}'
    
# #     plt.savefig(f"output_data/{figname}.png")

# #     # Show plot
#     # plt.show()

# LRPlots(avinc, totcourses, "Average Income", "Total Courses")

In [ ]:
# scatter plot of state population vs number of disc golf courses

In [ ]:
# scatter plot of state average income vs number of disc golf courses

In [ ]:
# #My  Define a function to create Linear Regression plots
# # Columns ['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness','Wind Speed', 'Country', 'Date'],
# # Max Temp units (Celsius), Humidity units (%), Cloudiness units (%) Wind Speed (m/s)

# datasetID = 'DiscGolf'
# xparam = 'Lat'
# yparam = 'Wind Speed'
# yunits = '(m/s)'
# fcolor = 'lightblue'
# ecolor = 'mediumblue'

# #some color options mistyrose and maroon,seashell  and saddlebrown,honeydew and forestgreen,lightblue and meduimblue
# #https://matplotlib.org/stable/gallery/color/named_colors.html


# def LRPlots():
#     xdata = df[xparam]
#     ydata = df[yparam]

#     (slope, intercept, rvalue, pvalue, stderr) = linregress(xdata, ydata)
#     regress_values = xdata * slope + intercept
#     line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

#     plt.figure(figsize = (9,7))
#     plt.scatter(xdata, ydata, marker="o", facecolors=fcolor, edgecolors=ecolor,s =75)

#     # plt.xlim(-55,85)
#     # plt.ylim(0,105)
#     annotatex = min(xdata)+0.1*max(xdata)
#     annotatey = max(ydata)-0.1*max(ydata)

#     plt.plot(xdata,regress_values,"darkslategray")
#     plt.annotate(f'{line_eq} r={round(rvalue,2)}',(annotatex,annotatey),fontsize=15,color="black")
    

#     plt.title(f"City Latitude vs. {yparam} ({datedatapull})")
#     plt.xlabel("Latitude")
#     plt.ylabel(f"[{yparam} {yunits}]")
#     plt.grid()


    
#     print(f'The r-value is {round(rvalue,2)}')

   

#     # Save the figure
#     figname = f'{datasetID}_{xparam}_vs_{yparam}'
    
#     plt.savefig(f"output_data/{figname}.png")

#     # Show plot
#     plt.show()

# LRPlots()

In [ ]:
# #Disc Golf update zip so dtype is int64 instead of object so we can merge the two dfs

# discgolfdf2 = discgolfdf.dropna(subset = ['zip'])
# discgolfdf2sort = discgolfdf2.sort_values('zip')
# discgolfdf2sort

In [ ]:
# discgolf drop zip NaN  DROP NA DOESN'T WORK BECAUSE IT'S NOT NA,
#    sort doesn't work because it doesn't sort them at the beginning or end.  
# DiscGolfClean = manually edited disc golf csv to fix zips,
#    added Modified column (no,yes) and Comment column (none, or modification info)

In [ ]:
# discgolfcleandf.head()

In [ ]:
# discgolfcleandf.dtypes

In [ ]:
# #check zipcodecsv 
# zipcodedf.head(2)

In [ ]:
# zipcodedf.dtypes

In [ ]:
# #Create DraftMerge - inner join DiscGolfcleandf and zipcodedf
# #nice reference https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/
# draft_dgolf_avincome = pd.merge(discgolfcleandf,zipcodedf,how = 'inner',left_on='zip',right_on='zipcode')
# draft_dgolf_avincome.head()

In [ ]:
# #export draft_dgolf_avincome to csv
# draft_dgolf_avincome.to_csv('Resources/draft_dgolf_avincome.csv')

# Paul's Graphs

In [ ]:
# Create a mini Data Frame that contains the Name of the County, Median Income (for that county),
# Population Density (for that county), and Total Courses (in that county)
#
#    for county in area n_courses.sum()
#
# the mini data frame will have each county in the midwest region (12 states) in the rows
# and 4 columns: area_name, median_income, population_density, and total_courses

In [ ]:
# Create Scatter Plot Median Income (county) vs Total Courses (county)
# Linear Regression line (x = median income, y = total courses)

In [ ]:
# Create Scatter Plot Population Density (county) vs Total Courses (county)
# Linear Regression line (x = population density, y = total courses)